In [1]:
import pandas as pd
from django.db.models import F
from project.models import Project
from public_data.models import CommunePop

project = Project.objects.all().first()

In [4]:
headers = [
    "Commune",
    "Insee",
    "EPCI",
    "SCoT",
    "Département",
    "Région",
    "Année",
    "Changement",
    "Total",
]
qs = (
    CommunePop.objects.filter(city__in=project.cities.all())
    .annotate(
        Commune=F("city__name"),
        Insee=F("city__insee"),
        EPCI=F("city__epci__name"),
        SCoT=F("city__scot__name"),
        Département=F("city__departement__name"),
        Région=F("city__departement__region__name"),
        Année=F("year"),
        Changement=F("pop_change"),
        Total=F("pop"),
    )
    .values(*headers)
)
qs[:10]

<QuerySet [{'Commune': 'Antras', 'Insee': '32003', 'EPCI': 'CA Grand Auch Cœur de Gascogne', 'SCoT': None, 'Département': 'Gers', 'Région': 'Occitanie', 'Année': 2019, 'Changement': -1, 'Total': 46}, {'Commune': 'Antras', 'Insee': '32003', 'EPCI': 'CA Grand Auch Cœur de Gascogne', 'SCoT': None, 'Département': 'Gers', 'Région': 'Occitanie', 'Année': 2018, 'Changement': 0, 'Total': 47}, {'Commune': 'Antras', 'Insee': '32003', 'EPCI': 'CA Grand Auch Cœur de Gascogne', 'SCoT': None, 'Département': 'Gers', 'Région': 'Occitanie', 'Année': 2017, 'Changement': -1, 'Total': 47}, {'Commune': 'Antras', 'Insee': '32003', 'EPCI': 'CA Grand Auch Cœur de Gascogne', 'SCoT': None, 'Département': 'Gers', 'Région': 'Occitanie', 'Année': 2016, 'Changement': -1, 'Total': 48}, {'Commune': 'Antras', 'Insee': '32003', 'EPCI': 'CA Grand Auch Cœur de Gascogne', 'SCoT': None, 'Département': 'Gers', 'Région': 'Occitanie', 'Année': 2015, 'Changement': -1, 'Total': 49}, {'Commune': 'Antras', 'Insee': '32003', 'EPCI

In [3]:
df = (
    pd.DataFrame(qs, columns=headers)
    .fillna(0.0)
    .pivot(columns=["Année"], index=headers[:6], values=["Changement", "Total"])
)
df

Changement  \
Année                                                                                      2006   
Commune              Insee EPCI                           SCoT Département Région                 
Antras               32003 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Auch                 32013 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Augnax               32014 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Auterive             32019 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Ayguetinte           32024 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Biran                32054 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Bonas                32059 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Castelnau-Barbarens  32076 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Castillon-Massas     32089 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Castin               32091 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Castéra-Verduzan     32083 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Crastes              32112 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Duran                32117 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Jegun                32162 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Lahitte              32183 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Lavardens            32204 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Leboulin             32207 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Mirepoix             32258 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Montaut-les-Créneaux 32279 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Montégut             32282 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Mérens               32251 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Nougaroulet          32298 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Ordan-Larroque       32301 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Pavie                32307 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Pessan               32312 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Peyrusse-Massas      32316 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Preignan             32331 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Puycasquier          32335 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Roquefort            32347 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Roquelaure           32348 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Saint-Jean-Poutge    32382 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Saint-Lary           32384 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Sainte-Christie      32368 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Tourrenquets         32453 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   

                                                                                             \
Année                                                                                  2007   
Commune              Insee EPCI                           SCoT Département Région             
Antras               32003 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie    0.0   
Auch                 32013 CA Grand Auch 

In [6]:
headers = [
    "Commune",
    "Insee",
    "EPCI",
    "SCoT",
    "Département",
    "Région",
    "Année",
    "Changement",
    "Total",
]
qs = (
    CommunePop.objects.filter(city__in=project.cities.all())
    .annotate(
        Commune=F("city__name"),
        Insee=F("city__insee"),
        EPCI=F("city__epci__name"),
        SCoT=F("city__scot__name"),
        Département=F("city__departement__name"),
        Région=F("city__departement__region__name"),
        Année=F("year"),
        Changement=F("household_change"),
        Total=F("household"),
    )
    .values(*headers)
)
df = (
    pd.DataFrame(qs, columns=headers)
    .fillna(0.0)
    .pivot(columns=["Année"], index=headers[:6], values=["Changement", "Total"])
)
df

Changement  \
Année                                                                                      2006   
Commune              Insee EPCI                           SCoT Département Région                 
Antras               32003 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Auch                 32013 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Augnax               32014 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Auterive             32019 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Ayguetinte           32024 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Biran                32054 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Bonas                32059 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Castelnau-Barbarens  32076 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Castillon-Massas     32089 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Castin               32091 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Castéra-Verduzan     32083 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Crastes              32112 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Duran                32117 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Jegun                32162 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Lahitte              32183 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Lavardens            32204 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Leboulin             32207 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Mirepoix             32258 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Montaut-les-Créneaux 32279 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Montégut             32282 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Mérens               32251 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Nougaroulet          32298 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Ordan-Larroque       32301 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Pavie                32307 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Pessan               32312 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Peyrusse-Massas      32316 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Preignan             32331 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Puycasquier          32335 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Roquefort            32347 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Roquelaure           32348 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Saint-Jean-Poutge    32382 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Saint-Lary           32384 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Sainte-Christie      32368 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   
Tourrenquets         32453 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie        0.0   

                                                                                           \
Année                                                                                2007   
Commune              Insee EPCI                           SCoT Département Région           
Antras               32003 CA Grand Auch Cœur de Gascogne 0.0  Gers        Occitanie  0.0   
Auch                 32013 CA Grand Auch Cœur de 